In [3]:
from road_segments import *
import os
from os import listdir
from os.path import isfile, join

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
request_url = build_url(gps_raw[:100])
result = requests.get(request_url, params=params)
response = result.json()

street_times        = get_street_times(response)
street_points       = get_street_points(street_times)
street_frame_bounds = get_street_bounds(street_times)

reloaded


In [9]:
method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)


pg = o3d.io.read_pose_graph(dataset + "/pose/full.json")

g2n = get_gps_to_node(pg, gps_raw, 0)

fullpg_idxmap = {}
street_g2pg   = {}

for street, bounds in street_frame_bounds.items():
    print("Working on {}".format(street))
    
    npgs, g2pg, idxmap = chunk_pg(pg, g2n, int(bounds['start']), int(bounds['stop']))
    
    street_g2pg[street] = g2pg
    fullpg_idxmap[street] = idxmap
    print("Got {} pose graphs".format(len(npgs)))
    
    
    for idx in range(len(npgs)):
        streetdir = dataset + "/pose/" + street.replace(" ", "_")
        
        if not os.path.isdir(streetdir):
            os.mkdir(streetdir)
            
        o3d.registration.global_optimization(npgs[idx], method, criteria, option)

        o3d.io.write_pose_graph(streetdir + "/{0:03}.json".format(idx), npgs[idx])
    

Working on Rigi Avenue
Got 13 pose graphs
Working on Coughlin Avenue
Got 3 pose graphs
Working on Colby Avenue
Got 11 pose graphs
Working on Tyson Place
Got 3 pose graphs


In [17]:
method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)

for street, nothing in street_times.items():
    streetdir = dataset + "/pose/" + street.replace(" ", "_")
    for fidx in range(len(street_g2pg[street])):

        print("{}: {}".format(street, fidx))
        road = o3d.io.read_pose_graph(streetdir + "/{0:03}.json".format(fidx))

        sp = street_g2pg[street][fidx]

        gps_start = np.array([sp[0][4],sp[0][5],1])
        gps_stop = np.array([sp[-1][4],sp[-1][5],1])
        gps_bearing = gps_stop - gps_start

        road_start = road.nodes[0].pose[:3,3]
        road_stop = road.nodes[len(road.nodes)-1].pose[:3,3]
        road_bearing = road_stop - road_start

        #camera travels in Z direction, need to rotate to align with gps bearing
        t = R.from_dcm(get_rotation(gps_bearing, road_bearing))

        gls = lineset_from_gps(sp, t)
        pls = lineset_from_nodes(road)

        inject_gps_edges(road, sp, fullpg_idxmap[street][fidx][0], t)
        o3d.io.write_pose_graph(streetdir + "/{0:03}_test.json".format(fidx), road)

        o3d.registration.global_optimization(road, method, criteria, option)
        o3d.io.write_pose_graph(streetdir + "/{0:03}_optimized_with_gps.json".format(fidx), road)


Rigi Avenue: 0
Rigi Avenue: 1
Rigi Avenue: 2
Rigi Avenue: 3
Rigi Avenue: 4
Rigi Avenue: 5
Rigi Avenue: 6
Rigi Avenue: 7
Rigi Avenue: 8
Rigi Avenue: 9
Rigi Avenue: 10
Rigi Avenue: 11
Rigi Avenue: 12
Coughlin Avenue: 0
Coughlin Avenue: 1
Coughlin Avenue: 2
Colby Avenue: 0
Colby Avenue: 1
Colby Avenue: 2
Colby Avenue: 3
Colby Avenue: 4
Colby Avenue: 5
Colby Avenue: 6
Colby Avenue: 7
Colby Avenue: 8
Colby Avenue: 9
Colby Avenue: 10
Tyson Place: 0
Tyson Place: 1
Tyson Place: 2
